In [1]:


''' 
A script to generate the initial list of intitutions: 

This is not to be used in production. 

Author: 

Dan Ellis
'''

' \nA script to generate the initial list of intitutions: \n\nThis is not to be used in production. \n\nAuthor: \n\nDan Ellis\n'

In [2]:
import requests,json
import urllib.parse

# Strip out strange characters and insert in the desired format
format_name = lambda n : urllib.parse.quote(n)
# Make the API call
url = 'https://api.ror.org/organizations?affiliation=%{}s'
debug = False


In [3]:


def ROR(name):

    response = requests.get(url.format(name))

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code} - {response.text}")



In [6]:
instiution_file = '../../../CMIP6_CVs/CMIP6_institution_id.json'
institution = json.load(open(instiution_file,'r')).get('institution_id')


In [7]:

results = {}

for k6,i62 in institution.items():

    for i6 in i62.split(';'):
    
        results[f'{k6}: {i6}'] = []
            
        
        i = ROR(f'{k6} {i6}').get('items')

        for item in i:
            # print ('\n\n\n')
        # item = i[0]
            organization = item['organization']
        
            name = organization['name']
            score = item['score']
            id = organization['id']
            established = organization['established']
            
            organization = item["organization"]
            aliases = item.get('aliases') or []
            acronyms = item.get('acronyms') or []

            
            addresses = organization["addresses"]
            addr = []
            for address in addresses:
                longitude = address["lng"]
                latitude = address["lat"]
                city = address["city"]
                state = address["state"]
                country = organization["country"]["country_name"]
                address_string = f"{city }, {state or ''}, {country} {address['state_code']or ''}"
                
                
                addr.append(dict(zip('lon lat address country city'.split(),[longitude,latitude,address_string,country,city])))


            # Extract the relationships
            relationships = []
            for relationship in organization['relationships']:
                relationship_name = relationship['label']
                relationship_url = relationship['id']
                relationships.append({'name': relationship_name, 'url': relationship_url})


            # # Print the extracted information
            # print("Key:", i6)
            # print("Name:", name)
            # print("AKA", aliases, acronyms)
            # print("Score:", score)
            # print("ID:", id)
            # print("Established:", established)
            # print('Addresses', addr)
            

            # print("Relationships:")
            # for relationship in relationships:
            #     print("- Name:", relationship['name'])
            #     print("  URL:", relationship['url'])

            dummy = {
                "Name": name,
                "Acronyms": acronyms,
                "ID": id,
                "Addresses": addr[0]['city'],
                # "Relationships": relationships
            }
        
            results[f'{k6}: {i6}'].append(dummy)



json.dump(results,open('./inst_results.json','w'), indent=4)

In [23]:
j = institution.copy()
for i in j:
    j[i]={'CMIP6key':i}
json.dump(results,open('./template.json','w'), indent=4)